<a href="https://colab.research.google.com/github/qjy9ey/qjy9ey/blob/main/Finance_API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
import requests

In [17]:
# Task 1: Fetch Basic Stock Data
stocks = input("Enter at least two stock symbols separated by commas: ")
querystring = {"symbols": stocks}
url = "https://yfapi.net/v6/finance/quote"
apikey = "" # Insert API key here.
headers = {'x-api-key': apikey}
response = requests.get(url, headers = headers, params = querystring)
data = response.json()['quoteResponse']['result']

for stock in data:
  print(f"Stock Ticker: {stock['symbol']}, Company: {stock.get('longName', 'N/A')}, Current Market Price: ${stock.get('regularMarketPrice', 'N/A')}")

Enter at least two stock symbols separated by commas: AAPL,MSFT
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36
Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $399.73


In [9]:
# Fetch Additional Data Using Modules
stock = input("Enter stock symbol: ")
module = input("Enter module (summaryDetail, financialData): ")
if module not in ["summaryDetail", "financialData"]:
    module = "summaryDetail"

url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
apikey = "" # Insert API key here.
headers = {'x-api-key': apikey}
querystring = {"modules": module}

response = requests.get(url, headers=headers, params=querystring)
data = response.json()

result = data.get('quoteSummary', {}).get('result', [{}])[0]

if module == "summaryDetail":
    high_52 = result.get('summaryDetail', {}).get('fiftyTwoWeekHigh', {}).get('raw', "N/A")
    low_52 = result.get('summaryDetail', {}).get('fiftyTwoWeekLow', {}).get('raw', "N/A")
    roa = "N/A"  # ROA is not available in summaryDetail.

elif module == "financialData":
    high_52 = "N/A"  # 52-week data is not available in financialData.
    low_52 = "N/A"
    roa = result.get('financialData', {}).get('returnOnAssets', {}).get('raw', "N/A")

df = pd.DataFrame({"52 Week High": [high_52], "52 Week Low": [low_52], "ROA": [roa]})
print(df.to_string(index=False))

Enter stock symbol: AAPL
Enter module (summaryDetail, financialData): summaryDetail
 52 Week High  52 Week Low ROA
        260.1       164.08 N/A


In [23]:
# Task 2.2: Fetch Additional Data Using Modules
url = "https://yfapi.net/v1/finance/trending/US"
api_key = ""
headers = {
    'x-api-key': apikey
}

response = requests.get(url, headers=headers)
data = response.json()

trending_tickers = [item['symbol'] for item in data['finance']['result'][0]['quotes']]

querystring = {"symbols": ",".join(trending_tickers)}

quote_url = "https://yfapi.net/v6/finance/quote"

response = requests.get(quote_url, headers=headers, params=querystring)
quote_data = response.json()

df = pd.DataFrame(quote_data['quoteResponse']['result'])

df = df[['symbol', 'shortName', 'regularMarketPrice', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow']]
df.columns = ['Ticker', 'Name', 'Current Price', '52-Week High', '52-Week Low']

print(df)

   Ticker                             Name  Current Price  52-Week High  \
0    NVDA               NVIDIA Corporation         131.28       153.130   
1     CRM                 Salesforce, Inc.         307.33       369.000   
2    SNOW                   Snowflake Inc.         166.19       194.400   
3    IONQ                       IonQ, Inc.          29.93        54.740   
4    BYND                Beyond Meat, Inc.           3.56        11.700   
5     NIO                         NIO Inc.           4.72         7.710   
6    MRNA                    Moderna, Inc.          33.58       170.470   
7    MARA              MARA Holdings, Inc.          12.45        30.710   
8    TDOC             Teladoc Health, Inc.          10.99        15.950   
9     APP             Applovin Corporation         331.00       525.150   
10   NTNX                    Nutanix, Inc.          69.35        75.800   
11     AI                      C3.ai, Inc.          26.44        45.080   
12    PBR  Petroleo Brasi

In [12]:
# Bonus Challenge: Elon’s Plane Location
api_key = "e3c212dc785ba9b1b621368f1fc21e5e"
planes = ["N628TS"]

for plane in planes:
    r = requests.get(f"http://api.aviationstack.com/v1/flights?access_key={api_key}&reg_number={plane}")
    d = r.json().get("data", [{}])[0]  # Get first (most recent) flight.

    departure = d.get("departure", {}).get("airport", "No departure data")
    arrival = d.get("arrival", {}).get("airport", "No arrival data")

    print(f"{plane}: {departure} → {arrival}")

N628TS: Jinjiang → Changzhou
